# Managing modeling complexity

In this notebook we highlight a few features of `opvious`'s [declarative modeling API](https://opvious.readthedocs.io/en/stable/modeling.html) which help build and maintain large models.

In [1]:
%pip install opvious

## Dependencies

It's good practice to split up large blocks of code into smaller meaningful units. This also holds for modeling code. To enable this, `Model` constructors accept an array of model dependencies. Each of these dependencies will automatically be included in the dependent's specification.

In [2]:
import opvious.modeling as om

class Environment(om.Model):
    """Parent model"""
    epochs = om.Dimension()
    active = om.Variable.indicator(epochs)
    
class Activity(om.Model):
    """Child model"""
    
    def __init__(self, environment):
        super().__init__(dependencies=[environment])  # Note the dependency
        self._environment = environment
        
    @om.objective
    def maximize_active_duration(self):
        return self._environment.active.total()

activity = Activity(Environment())

The child's specification will include all of its (potentially transitive) dependencies, organized by model:

<div class="alert alert-block alert-info">
    &#9432; In Jupyter notebooks, individual model specifications can be collapsed by clicking on the model's name.
</div>

In [3]:
activity.specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Activity</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^o_\mathrm{maximizeActiveDuration}&: \max \sum_{e \in E} \alpha_{e} \\
\end{align*}
$$
</div>
</details>

---

<details>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Environment</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{epochs}&: E \\
  \S^v_\mathrm{active}&: \alpha \in \{0, 1\}^{E} \\
\end{align*}
$$
</div>
</details>
</div>

`definition_counts` will also break down the definitions by model:

In [4]:
activity.definition_counts()

category,DIMENSION,OBJECTIVE,VARIABLE
title,,,
Activity,0,1,0
Environment,1,0,1


## Annotations

Calling a model's `specification` method performs basic validation when generating its output. Not all errors are detected however, for example mismatched indices would not be caught here but raise an exception later when solving.

Can you swap the error(s) in the model below?

In [5]:
class Invalid(om.Model):
    products = om.Dimension()
    stores = om.Dimension()
    inventory = om.Parameter.non_negative(products)
    shipment = om.Variable.natural(products, stores)
    
    @om.constraint
    def shipments_within_inventory(self):
        for p in self.products:
            yield om.total(self.shipment(s, p) for s in self.stores) <= self.inventory()

Invalid().specification()

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Invalid</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{products}&: P \\
  \S^d_\mathrm{stores}&: S \\
  \S^p_\mathrm{inventory}&: i \in \mathbb{R}_+^{P} \\
  \S^v_\mathrm{shipment}&: \sigma \in \mathbb{N}^{P \times S} \\
  \S^c_\mathrm{shipmentsWithinInventory}&: \forall p \in P, \sum_{s \in S} \sigma_{s,p} \leq i \\
\end{align*}
$$
</div>
</details>
</div>

Fortunately we can always use a client's [`annotate_specification`](https://opvious.readthedocs.io/en/stable/api-reference.html#opvious.Client.annotate_specification) method to discover all inconsistencies within a specification: mismatched indices, unexpected degree (e.g. quadratic expressions within a constraint), numeric operations on non-numeric terms, and much more. Any errors will be highlighted in orange in the specification and indicated by a warning symbol next to the model's name.

Let's try it on the specification above.

In [6]:
import opvious

client = opvious.Client.from_environment(default_endpoint=opvious.DEMO_ENDPOINT)

annotated = await client.annotate_specification(Invalid().specification())
annotated

<div style="margin-top: 1em; margin-bottom: 1em;">
<details open>
<summary style="cursor: pointer; text-decoration: underline; text-decoration-style: dotted;">Invalid &#9888;</summary>
<div style="margin-top: 1em;">
$$
\begin{align*}
  \S^d_\mathrm{products}&: P \\
  \S^d_\mathrm{stores}&: S \\
  \S^p_\mathrm{inventory}&: i \in \mathbb{R}_+^{P} \\
  \S^v_\mathrm{shipment}&: \sigma \in \mathbb{N}^{P \times S} \\
  \S^c_\mathrm{shipmentsWithinInventory}&: \forall p \in P, \sum_{s \in S} \sigma_{\color{orange}{s},\color{orange}{p}} \leq \color{orange}{i} \\
\end{align*}
$$
</div>
</details>
</div>

The 3 errors were correctly identified, all in the `shipmentsWithinInventory` constraint:

* $\sigma$'s indices both have incorrect types (they are swapped);
* $i$ is missing an index.

Any errors are also available for programmatic use via the specification's `annotation` property:

In [7]:
annotated.annotation.issues

{0: [LocalSpecificationIssue(source_index=0, start_offset=282, end_offset=282, message='This expression is not compatible with its context; please check that dimensions and numericity match', code='ERR_INCOMPATIBLE_VALUE'),
  LocalSpecificationIssue(source_index=0, start_offset=284, end_offset=284, message='This expression is not compatible with its context; please check that dimensions and numericity match', code='ERR_INCOMPATIBLE_VALUE'),
  LocalSpecificationIssue(source_index=0, start_offset=292, end_offset=292, message='This subscript has size 0 which does not match the underlying space (of rank 1)', code='ERR_INCOMPATIBLE_SUBSCRIPT')]}

## Fragments

Coming soon.